In [4]:
import torch, pickle
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import pandas as pd
from gensim.models import Word2Vec

In [11]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.o2o = nn.Linear(hidden_size + output_size, output_size)
        self.dropout = nn.Dropout(0.1)
        self.softmax = nn.LogSoftmax()

    def forward(self, input, hidden):
        input_combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(input_combined)
        output = self.i2o(input_combined)
        output_combined = torch.cat((hidden, output), 1)
        output = self.o2o(output_combined)
        output = self.dropout(output)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return Variable(torch.zeros(1, self.hidden_size))

In [5]:
# -- import embeddings
with open('../../data/lemmas_embeddings.pickle', 'rb') as f:
    data = pickle.load(f)

In [6]:
data.keys()

dict_keys(['embeddings', 'vocab'])

In [66]:
embeddings = data['embeddings'].astype(np.float32)
vocab = data['vocab']

In [12]:
# -- load lemma text
with open('../../data/lemmas.pickle', 'rb') as g:
    lemma = pickle.load(g)

In [14]:
# lemma[0]

In [74]:
EMBEDDING_DIM = 100
HIDDEN_DIM = 100
rnn = RNN(EMBEDDING_DIM, HIDDEN_DIM, len(vocab))
criterion = nn.NLLLoss()
learning_rate = 0.00005

In [ ]:
hidden = rnn.initHidden()
loss = 0.0
for sentence in lemma[:1]:
    for i in range(len(sentence) - 1):
        try: # -- get rid of digits in vocab
            current_word = sentence[i]
            current_index = vocab[current_word]
            current_embedding = torch.from_numpy(embeddings[current_index]).unsqueeze(0)
            vembedding = Variable(current_embedding)
            next_word = torch.from_numpy(np.array([vocab[sentence[i+1]]]))
            output, hidden = rnn(vembedding, hidden)

            loss += criterion(output, Variable(next_word))

        except KeyError as e:
            print('I got a KeyError - reason "%s"' % str(e))

loss.backward()
for p in rnn.parameters():
    p.data.add_(-learning_rate, p.grad.data)

I got a KeyError - reason "'02:28:01'"
I got a KeyError - reason "'02:28:01'"
I got a KeyError - reason "'06:28:01'"
I got a KeyError - reason "'06:28:01'"
I got a KeyError - reason "'69,197'"
I got a KeyError - reason "'69,197'"
I got a KeyError - reason "'18,222'"
I got a KeyError - reason "'18,222'"
